## This code implements an Actor-Critic Framework for the Cart Pole example provided by Open AI Gym.
  

The code is based on the Pytorch Examples (https://github.com/pytorch/examples)

In [ ]:
# Loading Libraries
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [ ]:
seed=1234
# Making the cartpole environment
env = gym.make('CartPole-v1')
env.seed(seed)
torch.manual_seed(seed)


In [ ]:
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])

In [ ]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.layer1 = nn.Linear(4, 128)
        self.action_head = nn.Linear(128, 2)
        self.value_head = nn.Linear(128, 1)

        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        x = F.relu(self.layer1(x))
        action_scores = self.action_head(x)
        state_values = self.value_head(x)
        return F.softmax(action_scores, dim=-1), state_values

In [ ]:
model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()

In [ ]:
def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)
    m = Categorical(probs)
    action = m.sample()
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
    return action.item()


def finish_episode():
    R = 0
    gamma=0.99
    saved_actions = model.saved_actions
    policy_losses = []
    value_losses = []
    rewards = []
    for r in model.rewards[::-1]:
        R = r + gamma * R
        rewards.insert(0, R)
    rewards = torch.tensor(rewards)
    rewards = (rewards - rewards.mean()) / (rewards.std() + eps)
    for (log_prob, value), r in zip(saved_actions, rewards):
        reward = r - value.item()
        policy_losses.append(-log_prob * reward)
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([r])))
    optimizer.zero_grad()
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()
    loss.backward()
    optimizer.step()
    del model.rewards[:]
    del model.saved_actions[:]


In [ ]:
render=False
running_reward = 10
log_interval=100
for i_episode in range(1,1001):
    state = env.reset()
    for t in range(10000): 
        action = select_action(state)
        state, reward, done, _ = env.step(action)
        if render:
            env.render()
        model.rewards.append(reward)
        if done:
          break

    running_reward = running_reward * 0.99 + t * 0.01
    finish_episode()
    if i_episode % log_interval == 0:
        print('Episode {}\t Time steps: {:5d} \t Running Reward: {:.2f}'.format(i_episode,t, running_reward))
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and the last episode runs to {} time steps!".format(running_reward, t))
            break


Episode 100	 Time steps:   135 	 Running Reward: 83.58
Episode 200	 Time steps:   466 	 Running Reward: 138.76
Episode 300	 Time steps:   499 	 Running Reward: 337.70
Episode 400	 Time steps:   438 	 Running Reward: 411.42
Episode 500	 Time steps:   499 	 Running Reward: 452.34
Episode 600	 Time steps:   499 	 Running Reward: 472.21
Episode 700	 Time steps:   499 	 Running Reward: 489.19
Solved! Running reward is now 489.19224801196765 and the last episode runs to 499 time steps!
